## Import

In [1]:
import selenium
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import requests
import time

In [3]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Korea_Investment_and_Securities\\scripts\\ipynb'

In [6]:
data_path = "C:\\Users\\PC0\\Documents\\GitHub\\Korea_Investment_and_Securities\\datasets\\"

In [7]:
niche_df = pd.read_csv(data_path + "etf_niche_grouped_df2.csv")

In [8]:
niche_df.head()

,Theme,Symbol,stock_names,cnts,marketCap
0,5G,NVDA,NVIDIA Corporation,4,6.047336e+11
1,5G,NXPI,NXP Semiconductors NV,3,4.943163e+10
2,5G,AMD,"Advanced Micro Devices, Inc.",3,1.379936e+11
3,5G,MRVL,"Marvell Technology, Inc.",3,5.716745e+10
4,5G,NaN,"Amazon.com, Inc.",3,NaN


In [12]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

In [13]:
driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

In [14]:
# investion.com 호출 
driver.get('https://www.investing.com/')

In [15]:
# 검색 창에 키 값 보내기 
search_box = driver.find_element_by_xpath('/html/body/div[5]/header/div[2]/div/div[3]/div[1]/input') # 객체 잡기 
search_box.send_keys('Amazon.com, Inc.')
search_box.send_keys(Keys.ENTER)

In [16]:
### 팝업창 개수 확인 하기 #### 두번째 부터가 팝업창 현재의 경우 팝업이 뜨지 않음  
#### 팝업이 발생하는경우 제거하기 if 문 걸어서 
tabs = driver.window_handles
print(tabs)

['CDwindow-315F152DEB07C8A8BFFE67AA5112C527']


In [17]:
#### 만약 팝업이 있는 경우 팝업을 제거하는 코드 
# driver.switch_to_window(tabs[1])
# driver.close()

In [67]:
# 현재 페이지 긁기 
soup =BeautifulSoup(driver.page_source,'html.parser')

In [18]:
a = np.load(data_path + 'stock_names.npy')
a

array(['10x Genomics Inc Class A',
       '17 Education & Technology Group, Inc. Unsponsored ADR',
       '1847 Goedeker Inc', ..., 'trivago N.V. Sponsored ADR Class A',
       'uniQure N.V.', 'voestalpine AG'], dtype='<U123')

In [19]:
start_idx=0

In [20]:
ticker_lst = []
investing_lst=[]
detail_lst=[]
etf_com_lst = []

In [182]:
# start_idx=183

In [22]:
from tqdm import tqdm

In [36]:
#### 긁기 
for i in tqdm(range(start_idx,len(a))):
    
    # 검색 창에 키 값 보내기 
    search_box = driver.find_element_by_xpath('/html/body/div[5]/header/div[2]/div/div[3]/div[1]/input') # 객체 잡기 
    search_box.send_keys(a[i])
    click_box = driver.find_element_by_xpath('/html/body/div[5]/header/div[2]/div/div[3]/div[1]/label') # 클릭 선택
    click_box.click()
    
    time.sleep(2)

    # 현재 페이지 긁기 
    soup =BeautifulSoup(driver.page_source,'html.parser')
    
    try:
        if driver.find_element_by_xpath('//*[@id="fullColumn"]/div/div[2]/div[2]/h2'):
            Ticker = driver.find_element_by_xpath('//*[@id="fullColumn"]/div/div[2]/div[2]/div[1]/a[1]/span[2]')
            ticker = Ticker.text
           
            Invest = driver.find_element_by_xpath('//*[@id="fullColumn"]/div/div[2]/div[2]/div[1]/a[1]/span[3]')
            investing_name = Invest.text
            
            Detail = driver.find_element_by_xpath('//*[@id="fullColumn"]/div/div[2]/div[2]/div[1]/a[1]/span[4]')
            detail = Detail.text

            ticker_lst.append(ticker) 
            investing_lst.append(investing_name)
            detail_lst.append(detail)
            etf_com_lst.append(a[i])
    except :
            ticker_lst.append('None') 
            investing_lst.append('None')
            detail_lst.append('None')
            etf_com_lst.append(a[i])
            
        
    search_box = driver.find_element_by_xpath('/html/body/div[5]/header/div[2]/div/div[3]/div[1]/input') 
    search_box.clear()
    
    time.sleep(3)

    start_idx += 1

100%|████████████████████████████████████████████████████████████████████████████| 2843/2843 [6:03:31<00:00,  7.67s/it]


In [37]:
df= pd.DataFrame({'etf.com_names':etf_com_lst,
             'Ticker_names':ticker_lst,
             'investing_names':investing_lst,
             'details':detail_lst})
df

,etf.com_names,Ticker_names,investing_names,details
0,10x Genomics Inc Class A,None,None,None
1,"17 Education & Technology Group, Inc. Unsponso...",YQ,17 Education Technology Group Inc,Stock - NASDAQ equities
2,1847 Goedeker Inc,None,None,None
3,"1Life Healthcare, Inc.",None,None,None
4,1st Source Corporation,SRCE,1st Source Corporation,Stock - NASDAQ equities
...,...,...,...,...
10081,thyssenkrupp AG,TKAG,Thyssenkrupp AG O.N.,Stock - Xetra equities
10082,transcosmos Inc.,9715,Transcosmos Inc,Stock - Tokyo equities
10083,trivago N.V. Sponsored ADR Class A,None,None,None
10084,uniQure N.V.,None,None,None


In [38]:
df.to_csv(data_path + 'investing_com.csv',index=False)